# LIBRARIES

In [ ]:
# !pip install boruta

In [6]:
import os
import ast
import json
import math
import ccxt
import time
import pickle
import random
import numpy as np
import configparser
import pandas as pd
import google.cloud
import tweepy as tw
from pandas_gbq import gbq
import matplotlib.cm as cm
from boruta import BorutaPy
from datetime import datetime
from datetime import timedelta
import matplotlib.pyplot as plt
from google.cloud import bigquery
from sklearn import preprocessing
from requests import Request, Session
from IPython.display import clear_output
from sklearn.metrics import confusion_matrix
from scipy.interpolate import UnivariateSpline
from sklearn.ensemble import BaggingClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV
from requests.exceptions import ConnectionError, Timeout, TooManyRedirects

#_____

pd.set_option('display.max_rows', 5000)
pd.options.mode.chained_assignment = None

# FUNCTIONS

In [7]:
#FUNCIÓN QUE LEE EL ARCHIVO CONFIG
def get_config(category, key):
    
    global config
    return config[category][key]

#_____

#FUNCIÓN QUE GENERA CONEXIÓN CON ARCHIVO CONFIG
def updateConfig(config_name):
    
    global config
    
    config = configparser.ConfigParser()
    config.sections()
    config.read(config_name)
    
#_____

#FUNCIÓN QUE GRAFICA POR COLORES SEGUN COMPRAS Y NO COMRPAS
def plot_colourline(x,y,c):
    c = c
    ax = plt.gca()
    for i in np.arange(len(x)-1):
        ax.plot([x[i],x[i+1]], [y[i],y[i+1]], c=c[i])
    return

#_____

#FUNCIÓN QUE ESCRIBE PICKLE DE VARIABLE
def writePickleVariable(variable,variable_name):
    pickle_out = open(variable_name+".pickle","wb")
    pickle.dump(variable, pickle_out)
    pickle_out.close()
    
#_____
    
#FUNCIÓN QUE LEE PICKLE DE VARIABLE
def readPickleVariable(variable_name):    
    return pickle.load(open(variable_name+".pickle","rb"))

#_____

#FUNCIÓN QUE REDONDEA FLOAT HACIA ABAJO SEGÚN DECIMALES
def round_decimals_down(number:float, decimals:int=2):
    """
    Returns a value rounded down to a specific number of decimal places.
    """
    if not isinstance(decimals, int):
        raise TypeError("decimal places must be an integer")
    elif decimals < 0:
        raise ValueError("decimal places has to be 0 or more")
    elif decimals == 0:
        return math.floor(number)

    factor = 10 ** decimals
    return math.floor(number * factor) / factor

#_____

#FUNCIÓN QUE REDONDEA FLOAT HACIA ARRIBA SEGÚN DECIMALES
def round_decimals_up(number:float, decimals:int=2):
    """
    Returns a value rounded down to a specific number of decimal places.
    """
    if not isinstance(decimals, int):
        raise TypeError("decimal places must be an integer")
    elif decimals < 0:
        raise ValueError("decimal places has to be 0 or more")
    elif decimals == 0:
        return math.ceil(number)

    factor = 10 ** decimals
    return math.ceil(number * factor) / factor

#_____

#FUNCIÓN QUE DESCARGA BASE DE DATOS DE MERCADO DE BIGQUERY
def downloadDataBaseBigQuery(market_model,time,rows=None):
    
    #_____GOOGLE CLOUD CONECTION
    os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="/home/ubuntu/SurfNet/predictionModels/PREDICTION/bigQueryAccess.json"

    #_____SI NO SE DA UN NÚMERO ESPECÍFICO DE FILAS POR PARÁMETRO
    if rows==None:

        #_____DESCARGAR BASE DE DATOS
        candlesDataBase_BigQuery=gbq.read_gbq("SELECT * FROM [dogwood-terra-308100:surfNet."+market_model.replace("/","_")+"] ORDER BY TIME DESC",project_id="dogwood-terra-308100",dialect="legacy").sort_values(by="TIME")
        candlesDataBase_BigQuery.reset_index(inplace=True,drop=True)

    else:

        #_____DESCARGAR BASE DE DATOS
        candlesDataBase_BigQuery=gbq.read_gbq("SELECT * FROM [dogwood-terra-308100:surfNet."+market_model.replace("/","_")+"] ORDER BY TIME DESC LIMIT "+str(rows),project_id="dogwood-terra-308100",dialect="legacy").sort_values(by="TIME")
        candlesDataBase_BigQuery.reset_index(inplace=True,drop=True)

    candlesDataBase_BigQuery.OPEN=candlesDataBase_BigQuery.OPEN.astype(float)
    candlesDataBase_BigQuery.HIGH=candlesDataBase_BigQuery.HIGH.astype(float)
    candlesDataBase_BigQuery.LOW=candlesDataBase_BigQuery.LOW.astype(float)
    candlesDataBase_BigQuery.CLOSE=candlesDataBase_BigQuery.CLOSE.astype(float)
    candlesDataBase_BigQuery.VOLUME=candlesDataBase_BigQuery.VOLUME.astype(float)

    #_____REESTRUCTURAR BASE DE DATOS
    candlesDataBase_BigQuery.reset_index(inplace=True,drop=True)

    if time != "1":

        #_____UPDATE CONFIG PREDICTION
        updateConfig("predictionModels.ini")

        #_____CREAR COLUMNAS
        candlesDataBase_BigQuery_copy=candlesDataBase_BigQuery.copy()
        candlesDataBase_BigQuery_copy=candlesDataBase_BigQuery_copy.set_index('TIME')
        candlesDataBase_BigQuery_copy_total=candlesDataBase_BigQuery_copy[["ID"]].groupby(pd.Grouper(freq=time+'min', offset=time+'min')).min()
        candlesDataBase_BigQuery_copy_total=candlesDataBase_BigQuery_copy_total.join(candlesDataBase_BigQuery_copy[["VOLUME"]].groupby(pd.Grouper(freq=time+'min', offset=time+'min')).sum())
        candlesDataBase_BigQuery_copy_total=candlesDataBase_BigQuery_copy_total.join(candlesDataBase_BigQuery_copy[["LOW"]].groupby(pd.Grouper(freq=time+'min', offset=time+'min')).min())
        candlesDataBase_BigQuery_copy_total=candlesDataBase_BigQuery_copy_total.join(candlesDataBase_BigQuery_copy[["HIGH"]].groupby(pd.Grouper(freq=time+'min', offset=time+'min')).max())

        #_____AGREAR DATOS
        for i in list(candlesDataBase_BigQuery_copy_total.index.values):
            try:
                candlesDataBase_BigQuery_copy_total.at[i,"OPEN"]=candlesDataBase_BigQuery.loc[candlesDataBase_BigQuery.TIME==i]["OPEN"].values[0]
                final_time = i + np.timedelta64(int(time),'m')
                if final_time in list(candlesDataBase_BigQuery_copy_total.index.values):
                    candlesDataBase_BigQuery_copy_total.at[i,"CLOSE"]=candlesDataBase_BigQuery.loc[candlesDataBase_BigQuery.TIME==final_time]["OPEN"].values[0]
                else:
                    candlesDataBase_BigQuery_copy_total.at[i,"CLOSE"]=candlesDataBase_BigQuery.at[len(candlesDataBase_BigQuery)-1,"CLOSE"]
            except:
                pass

        #_____RESET INDEX
        candlesDataBase_BigQuery_copy_total.reset_index(level=['TIME'],inplace=True)

        #_____COLUMNA MARKET
        for i in range(0,len(candlesDataBase_BigQuery_copy_total)):
            candlesDataBase_BigQuery_copy_total.at[i,"MARKET"]=candlesDataBase_BigQuery.MARKET.values[0]

        #_____ORDENAR COLUMNAS
        column_names = list(candlesDataBase_BigQuery.columns)
        candlesDataBase_BigQuery_copy_total = candlesDataBase_BigQuery_copy_total.reindex(columns=column_names)
        candlesDataBase_BigQuery_copy_total.dropna(inplace=True)
        candlesDataBase_BigQuery_copy_total.reset_index(inplace=True,drop=True)
        candlesDataBase_BigQuery=candlesDataBase_BigQuery_copy_total.copy()

    #_____RETURN
    return candlesDataBase_BigQuery

#_____

#FUNCIÓN QUE DEVUELVE EL RSI DADO UN PERIODO DETERMINADO
def RSI(candlesDataBase_BigQuery):
    
    global config
    updateConfig("predictionModels.ini")
    
    series=pd.Series(list(candlesDataBase_BigQuery.CLOSE.values))
    period=int(get_config("PARAMETERS","RSI_LEN"))
    
    delta = series.diff().dropna()
    u = delta * 0
    d = u.copy()
    u[delta > 0] = delta[delta > 0]
    d[delta < 0] = -delta[delta < 0]
    u[u.index[period-1]] = np.mean( u[:period] ) #first value is sum of avg gains
    u = u.drop(u.index[:(period-1)])
    d[d.index[period-1]] = np.mean( d[:period] ) #first value is sum of avg losses
    d = d.drop(d.index[:(period-1)])
    rs = pd.DataFrame.ewm(u, com=period-1, adjust=False).mean() / \
         pd.DataFrame.ewm(d, com=period-1, adjust=False).mean()
    
    rsi=100 - 100 / (1 + rs)
    
    for i in list(rsi.index.values):
        candlesDataBase_BigQuery.at[i,"RSI"]=rsi[i]
                                        
    return candlesDataBase_BigQuery

#_____

#FUNCIÓN QUE DEVUELVE EL RSI DADO UN PERIODO DETERMINADO
def RSI_GENERAL(series, period):
    delta = series.diff().dropna()
    u = delta * 0
    d = u.copy()
    u[delta > 0] = delta[delta > 0]
    d[delta < 0] = -delta[delta < 0]
    u[u.index[period-1]] = np.mean( u[:period] ) #first value is sum of avg gains
    u = u.drop(u.index[:(period-1)])
    d[d.index[period-1]] = np.mean( d[:period] ) #first value is sum of avg losses
    d = d.drop(d.index[:(period-1)])
    rs = pd.DataFrame.ewm(u, com=period-1, adjust=False).mean() / \
         pd.DataFrame.ewm(d, com=period-1, adjust=False).mean()
    return 100 - 100 / (1 + rs)

#_____

#FUNCIÓN QUE CALCULA MACD
def MACD(candlesDataBase_BigQuery):
    
    global config
    updateConfig("predictionModels.ini")
    
    close_data_df=candlesDataBase_BigQuery.copy()
    close_data_df.CLOSE=close_data_df.CLOSE.astype(float)
    close_data_df=close_data_df[["CLOSE"]]
    
    #_____CREAR VARIABLE MACD
    macd_12 = close_data_df.ewm(span=int(get_config("PARAMETERS","MACD_SHORT")), adjust=False).mean()
    macd_26 = close_data_df.ewm(span=int(get_config("PARAMETERS","MACD_LONG")), adjust=False).mean()
    macd = macd_12 - macd_26
    macd_9 = macd.ewm(span=int(get_config("PARAMETERS","MACD_TRIGGER")), adjust=False).mean()

    #_____AGREGAR VARIABLES A LA BASE DE DATOS
    candlesDataBase_BigQuery["MACD"]=macd
    #candlesDataBase_BigQuery["MACD_"+get_config("PARAMETERS","MACD_SHORT")]=macd_12
    #candlesDataBase_BigQuery["MACD_"+get_config("PARAMETERS","MACD_LONG")]=macd_26
    candlesDataBase_BigQuery["MACD_TRIGGER"]=macd_9
    
    return candlesDataBase_BigQuery

#_____

#FUNCIÓN QUE BRINDA LOS NIVELES DE FIBONACCI
def fibonacciLevels(candlesDataBase_BigQuery):
    
    global config
    global fractal
    
    #_____UPDATE CONFIG PREDICTION
    updateConfig("predictionModels.ini")
    
    #_____ACTUALIZAR PARÁMETROS
    FIBONACCI_PERIOD=int(int(get_config("PARAMETERS","FIBONACCI_PERIOD"))/int(fractal))

    #_____RECORRER DATOS DE TEST
    for i in range(FIBONACCI_PERIOD,len(candlesDataBase_BigQuery)):
        
        #_____FIBONACCI
        lows=list(candlesDataBase_BigQuery[i-FIBONACCI_PERIOD:i].LOW.values)
        highs=list(candlesDataBase_BigQuery[i-FIBONACCI_PERIOD:i].HIGH.values)
        Price_Min = min(lows)
        Price_Max = max(highs)
        Diff = Price_Max-Price_Min
        level1 = Price_Max - 0.236 * Diff
        level2 = Price_Max - 0.382 * Diff
        level3 = Price_Max - 0.618 * Diff
        level4 = Price_Max - 0.786 * Diff
        
        candlesDataBase_BigQuery.at[i,"FIBO_LEVEL_1"]=level1
        candlesDataBase_BigQuery.at[i,"FIBO_LEVEL_2"]=level2
        candlesDataBase_BigQuery.at[i,"FIBO_LEVEL_3"]=level3
        candlesDataBase_BigQuery.at[i,"FIBO_LEVEL_4"]=level4
            
    return candlesDataBase_BigQuery

#_____

#FUNCIÓN QUE CREA VARIABLES ENCONTRADAS EN ARTÍCULOS
def newVariables(candlesDataBase_BigQuery):
    
    #_____VARIABLES NUEVAS
    candlesDataBase_BigQuery['SMA5_VOL'] = candlesDataBase_BigQuery.groupby('MARKET')['VOLUME'].transform(lambda x: x.rolling(window = 5).mean())
    candlesDataBase_BigQuery['SMA15_VOL'] = candlesDataBase_BigQuery.groupby('MARKET')['VOLUME'].transform(lambda x: x.rolling(window = 15).mean())
    candlesDataBase_BigQuery['SMA_VOL_RATIO'] = candlesDataBase_BigQuery['SMA5_VOL']/candlesDataBase_BigQuery['SMA15_VOL']

    #_____VARIABLES NUEVAS
    candlesDataBase_BigQuery['LOWEST_5D'] = candlesDataBase_BigQuery.groupby('MARKET')['LOW'].transform(lambda x: x.rolling(window = 5).min())
    candlesDataBase_BigQuery['HIGH_5D'] = candlesDataBase_BigQuery.groupby('MARKET')['HIGH'].transform(lambda x: x.rolling(window = 5).max())
    candlesDataBase_BigQuery['LOWEST_15D'] = candlesDataBase_BigQuery.groupby('MARKET')['LOW'].transform(lambda x: x.rolling(window = 15).min())
    candlesDataBase_BigQuery['HIGH_15D'] = candlesDataBase_BigQuery.groupby('MARKET')['HIGH'].transform(lambda x: x.rolling(window = 15).max())
    candlesDataBase_BigQuery['STOCHASTIC_5'] = ((candlesDataBase_BigQuery['CLOSE'] - candlesDataBase_BigQuery['LOWEST_5D'])/(candlesDataBase_BigQuery['HIGH_5D'] - candlesDataBase_BigQuery['LOWEST_5D']))*100
    candlesDataBase_BigQuery['STOCHASTIC_15'] = ((candlesDataBase_BigQuery['CLOSE'] - candlesDataBase_BigQuery['LOWEST_15D'])/(candlesDataBase_BigQuery['HIGH_15D'] - candlesDataBase_BigQuery['LOWEST_15D']))*100
    candlesDataBase_BigQuery['STOCHASTIC_D_5'] = candlesDataBase_BigQuery['STOCHASTIC_5'].rolling(window = 5).mean()
    candlesDataBase_BigQuery['STOCHASTIC_D_15'] = candlesDataBase_BigQuery['STOCHASTIC_5'].rolling(window = 15).mean()
    candlesDataBase_BigQuery['STOCHASTIC_RATIO'] = candlesDataBase_BigQuery['STOCHASTIC_D_5']/candlesDataBase_BigQuery['STOCHASTIC_D_15']
    
    return candlesDataBase_BigQuery

#_____

#FUNCIÓN SUPERTREND
def superTrend(df, atr_period, multiplier):
    
    high = df['HIGH']
    low = df['LOW']
    close = df['CLOSE']
    
    # calculate ATR
    price_diffs = [high - low, 
                   high - close.shift(), 
                   close.shift() - low]
    true_range = pd.concat(price_diffs, axis=1)
    true_range = true_range.abs().max(axis=1)
    # default ATR calculation in supertrend indicator
    atr = true_range.ewm(alpha=1/atr_period,min_periods=atr_period).mean() 
    # df['atr'] = df['tr'].rolling(atr_period).mean()
    
    # HL2 is simply the average of high and low prices
    hl2 = (high + low) / 2
    # upperband and lowerband calculation
    # notice that final bands are set to be equal to the respective bands
    final_upperband = upperband = hl2 + (multiplier * atr)
    final_lowerband = lowerband = hl2 - (multiplier * atr)
    
    # initialize Supertrend column to True
    supertrend = [True] * len(df)
    
    for i in range(1, len(df.index)):
        curr, prev = i, i-1
        
        # if current close price crosses above upperband
        if close[curr] > final_upperband[prev]:
            supertrend[curr] = True
        # if current close price crosses below lowerband
        elif close[curr] < final_lowerband[prev]:
            supertrend[curr] = False
        # else, the trend continues
        else:
            supertrend[curr] = supertrend[prev]
            
            # adjustment to the final bands
            if supertrend[curr] == True and final_lowerband[curr] < final_lowerband[prev]:
                final_lowerband[curr] = final_lowerband[prev]
            if supertrend[curr] == False and final_upperband[curr] > final_upperband[prev]:
                final_upperband[curr] = final_upperband[prev]

        # to remove bands according to the trend direction
        if supertrend[curr] == True:
            final_upperband[curr] = np.nan
        else:
            final_lowerband[curr] = np.nan
    
    #return pd.DataFrame({
    #    'Supertrend': supertrend,
    #    'Final Lowerband': final_lowerband,
    #    'Final Upperband': final_upperband
    #}, index=df.index)

    df["SUPERTREND"]=supertrend

    return df

#_____

def suaviPlusDerivates(candlesDataBase_BigQuery):
    
    global config

    #_____UPDATE CONFIG PREDICTION
    updateConfig("predictionModels.ini")

    short_ma_hyper=int(get_config("PARAMETERS","SHORT_MA_HYPER"))
    short_ma_normal=int(get_config("PARAMETERS","SHORT_MA_NORMAL"))
    long_ma_normal=int(get_config("PARAMETERS","LONG_MA_NORMAL"))
    long_ma_hyper=int(get_config("PARAMETERS","LONG_MA_HYPER"))
    ultra_long_ma_normal=int(get_config("PARAMETERS","ULTRA_LONG_MA_NORMAL"))

    #_____VECTOR DE PRECIOS
    close_data_df=candlesDataBase_BigQuery.copy()
    close_data_df.CLOSE=close_data_df.CLOSE.astype(float)
    close_data_df=close_data_df[["CLOSE"]]

    #_____SUAVIZACIONES MA & EMA
    close_rolling_short_ma_hyper=close_data_df.rolling(short_ma_hyper).mean()
    close_rolling_short_ma_normal=close_data_df.rolling(short_ma_normal).mean()
    close_rolling_long_ma_normal=close_data_df.ewm(span=long_ma_normal).mean()
    close_rolling_long_ma_hyper=close_data_df.ewm(span=long_ma_hyper).mean()
    close_rolling_ultra_long_ma_normal=close_data_df.ewm(span=ultra_long_ma_normal).mean()

    #_____INSERTAR NUEVAS VARIABLES
    candlesDataBase_BigQuery["ROLLING_SHORT_MA_HYPER"]=close_rolling_short_ma_hyper
    candlesDataBase_BigQuery["ROLLING_SHORT_MA_NORMAL"]=close_rolling_short_ma_normal
    candlesDataBase_BigQuery["ROLLING_LONG_MA_NORMAL"]=close_rolling_long_ma_normal
    candlesDataBase_BigQuery["ROLLING_LONG_MA_HYPER"]=close_rolling_long_ma_hyper
    candlesDataBase_BigQuery["ROLLING_ULTRA_LONG_MA_NORMAL"]=close_rolling_ultra_long_ma_normal

    #_____BOLLINGER
    candlesDataBase_BigQuery['15MA'] = candlesDataBase_BigQuery.groupby('MARKET')['CLOSE'].transform(lambda x: x.rolling(window=15).mean())
    candlesDataBase_BigQuery['SD'] = candlesDataBase_BigQuery.groupby('MARKET')['CLOSE'].transform(lambda x: x.rolling(window=15).std())
    candlesDataBase_BigQuery['UPPERBAND'] = candlesDataBase_BigQuery['15MA'] + 2*candlesDataBase_BigQuery['SD']
    candlesDataBase_BigQuery['LOWERBAND'] = candlesDataBase_BigQuery['15MA'] - 2*candlesDataBase_BigQuery['SD']

    #_____RATE CHANGE
    candlesDataBase_BigQuery['RC'] = candlesDataBase_BigQuery.groupby('MARKET')['CLOSE'].transform(lambda x: x.pct_change(periods = 15))

    #_____ELIMINAR DATOS FALTANTES
    candlesDataBase_BigQuery.dropna(inplace=True)
    candlesDataBase_BigQuery.reset_index(inplace=True,drop=True)

    #####

    #_____CÁLCULO DE LA PRIMERA DERIVADA
    x_CLOSE= np.arange(len(candlesDataBase_BigQuery))
    y_spl_CLOSE = UnivariateSpline(x_CLOSE,candlesDataBase_BigQuery[["CLOSE"]],s=0,k=3)
    Y_1D_CLOSE=y_spl_CLOSE.derivative(n=1)
    Y_1D_CLOSE=pd.DataFrame(Y_1D_CLOSE(x_CLOSE))
    candlesDataBase_BigQuery["Y_1D_CLOSE"]=Y_1D_CLOSE

    #_____CÁLCULO DE LA PRIMERA DERIVADA
    x_ROLLING_SHORT_MA_HYPER= np.arange(len(candlesDataBase_BigQuery))
    y_spl_ROLLING_SHORT_MA_HYPER = UnivariateSpline(x_ROLLING_SHORT_MA_HYPER,candlesDataBase_BigQuery[["ROLLING_SHORT_MA_HYPER"]],s=0,k=3)
    Y_1D_ROLLING_SHORT_MA_HYPER=y_spl_ROLLING_SHORT_MA_HYPER.derivative(n=1)
    Y_1D_ROLLING_SHORT_MA_HYPER=pd.DataFrame(Y_1D_ROLLING_SHORT_MA_HYPER(x_ROLLING_SHORT_MA_HYPER))
    candlesDataBase_BigQuery["Y_1D_ROLLING_SHORT_MA_HYPER"]=Y_1D_ROLLING_SHORT_MA_HYPER

    #_____CÁLCULO DE LA PRIMERA DERIVADA
    x_ROLLING_SHORT_MA_NORMAL= np.arange(len(candlesDataBase_BigQuery))
    y_spl_ROLLING_SHORT_MA_NORMAL = UnivariateSpline(x_ROLLING_SHORT_MA_NORMAL,candlesDataBase_BigQuery[["ROLLING_SHORT_MA_NORMAL"]],s=0,k=3)
    Y_1D_ROLLING_SHORT_MA_NORMAL=y_spl_ROLLING_SHORT_MA_NORMAL.derivative(n=1)
    Y_1D_ROLLING_SHORT_MA_NORMAL=pd.DataFrame(Y_1D_ROLLING_SHORT_MA_NORMAL(x_ROLLING_SHORT_MA_NORMAL))
    candlesDataBase_BigQuery["Y_1D_ROLLING_SHORT_MA_NORMAL"]=Y_1D_ROLLING_SHORT_MA_NORMAL

    #_____CÁLCULO DE LA PRIMERA DERIVADA
    x_ROLLING_LONG_MA_NORMAL= np.arange(len(candlesDataBase_BigQuery))
    y_spl_ROLLING_LONG_MA_NORMAL = UnivariateSpline(x_ROLLING_LONG_MA_NORMAL,candlesDataBase_BigQuery[["ROLLING_LONG_MA_NORMAL"]],s=0,k=3)
    Y_1D_ROLLING_LONG_MA_NORMAL=y_spl_ROLLING_LONG_MA_NORMAL.derivative(n=1)
    Y_1D_ROLLING_LONG_MA_NORMAL=pd.DataFrame(Y_1D_ROLLING_LONG_MA_NORMAL(x_ROLLING_LONG_MA_NORMAL))
    candlesDataBase_BigQuery["Y_1D_ROLLING_LONG_MA_NORMAL"]=Y_1D_ROLLING_LONG_MA_NORMAL

    #_____CÁLCULO DE LA PRIMERA DERIVADA
    x_ROLLING_LONG_MA_HYPER= np.arange(len(candlesDataBase_BigQuery))
    y_spl_ROLLING_LONG_MA_HYPER = UnivariateSpline(x_ROLLING_LONG_MA_HYPER,candlesDataBase_BigQuery[["ROLLING_LONG_MA_HYPER"]],s=0,k=3)
    Y_1D_ROLLING_LONG_MA_HYPER=y_spl_ROLLING_LONG_MA_HYPER.derivative(n=1)
    Y_1D_ROLLING_LONG_MA_HYPER=pd.DataFrame(Y_1D_ROLLING_LONG_MA_HYPER(x_ROLLING_LONG_MA_HYPER))
    candlesDataBase_BigQuery["Y_1D_ROLLING_LONG_MA_HYPER"]=Y_1D_ROLLING_LONG_MA_HYPER

    #_____CÁLCULO DE LA PRIMERA DERIVADA
    x_ROLLING_ULTRA_LONG_MA_NORMAL= np.arange(len(candlesDataBase_BigQuery))
    y_spl_ROLLING_ULTRA_LONG_MA_NORMAL = UnivariateSpline(x_ROLLING_ULTRA_LONG_MA_NORMAL,candlesDataBase_BigQuery[["ROLLING_ULTRA_LONG_MA_NORMAL"]],s=0,k=3)
    Y_1D_ROLLING_ULTRA_LONG_MA_NORMAL=y_spl_ROLLING_ULTRA_LONG_MA_NORMAL.derivative(n=1)
    Y_1D_ROLLING_ULTRA_LONG_MA_NORMAL=pd.DataFrame(Y_1D_ROLLING_ULTRA_LONG_MA_NORMAL(x_ROLLING_ULTRA_LONG_MA_NORMAL))
    candlesDataBase_BigQuery["Y_1D_ROLLING_ULTRA_LONG_MA_NORMAL"]=Y_1D_ROLLING_ULTRA_LONG_MA_NORMAL

    #####

    #_____CALCULAR VARIABLE RSI
    RSI_ROLLING_SHORT_MA_HYPER = RSI_GENERAL(pd.Series(list(candlesDataBase_BigQuery.ROLLING_SHORT_MA_HYPER.values)), int(get_config("PARAMETERS","RSI_LEN")))
    RSI_ROLLING_SHORT_MA_NORMAL = RSI_GENERAL(pd.Series(list(candlesDataBase_BigQuery.ROLLING_SHORT_MA_NORMAL.values)), int(get_config("PARAMETERS","RSI_LEN")))
    RSI_ROLLING_LONG_MA_NORMAL = RSI_GENERAL(pd.Series(list(candlesDataBase_BigQuery.ROLLING_LONG_MA_NORMAL.values)), int(get_config("PARAMETERS","RSI_LEN")))
    RSI_ROLLING_LONG_MA_HYPER = RSI_GENERAL(pd.Series(list(candlesDataBase_BigQuery.ROLLING_LONG_MA_HYPER.values)), int(get_config("PARAMETERS","RSI_LEN")))
    RSI_ROLLING_ULTRA_LONG_MA_NORMAL = RSI_GENERAL(pd.Series(list(candlesDataBase_BigQuery.ROLLING_ULTRA_LONG_MA_NORMAL.values)), int(get_config("PARAMETERS","RSI_LEN")))

    #_____AGRAGAR VARIABLE RSI A BASE DE DATOS
    candlesDataBase_BigQuery["RSI_ROLLING_SHORT_MA_HYPER"]=RSI_ROLLING_SHORT_MA_HYPER
    candlesDataBase_BigQuery["RSI_ROLLING_SHORT_MA_NORMAL"]=RSI_ROLLING_SHORT_MA_NORMAL
    candlesDataBase_BigQuery["RSI_ROLLING_LONG_MA_NORMAL"]=RSI_ROLLING_LONG_MA_NORMAL
    candlesDataBase_BigQuery["RSI_ROLLING_LONG_MA_HYPER"]=RSI_ROLLING_LONG_MA_HYPER
    candlesDataBase_BigQuery["RSI_ROLLING_ULTRA_LONG_MA_NORMAL"]=RSI_ROLLING_ULTRA_LONG_MA_NORMAL

    #_____REESTRUCTURAR BASE DE DATOS
    candlesDataBase_BigQuery=candlesDataBase_BigQuery[int(get_config("PARAMETERS","ULTRA_LONG_MA_NORMAL"))+1:]
    candlesDataBase_BigQuery.reset_index(inplace=True,drop=True)
    
    return candlesDataBase_BigQuery

#_____

#FUNCIÓN QUE CALCULA MACD
def LAGS(candlesDataBase_BigQuery):
    
    for j in list(candlesDataBase_BigQuery.columns)[3:]:
        candlesDataBase_BigQuery[j+"_1"]=candlesDataBase_BigQuery[j].shift(1)
        candlesDataBase_BigQuery[j+"_2"]=candlesDataBase_BigQuery[j].shift(2)
        candlesDataBase_BigQuery[j+"_3"]=candlesDataBase_BigQuery[j].shift(3)
        
    candlesDataBase_BigQuery.dropna(inplace=True)
    candlesDataBase_BigQuery.reset_index(inplace=True,drop=True)
        
    return candlesDataBase_BigQuery

#_____

#CREATE VARIABLES
def variableCreation(candlesDataBase_BigQuery):
    
    global fractal
    updateConfig("predictionModels.ini")
    
    candlesDataBase_BigQuery=RSI(candlesDataBase_BigQuery)
    candlesDataBase_BigQuery=MACD(candlesDataBase_BigQuery)
    candlesDataBase_BigQuery=fibonacciLevels(candlesDataBase_BigQuery)
    candlesDataBase_BigQuery=newVariables(candlesDataBase_BigQuery)
    candlesDataBase_BigQuery=superTrend(candlesDataBase_BigQuery, int(get_config("PARAMETERS","ATR_PERIOD")), int(get_config("PARAMETERS","ATR_MULTIP")))
    candlesDataBase_BigQuery=suaviPlusDerivates(candlesDataBase_BigQuery)
    candlesDataBase_BigQuery=LAGS(candlesDataBase_BigQuery)
    
    candlesDataBase_BigQuery.dropna(inplace=True)
    candlesDataBase_BigQuery.reset_index(inplace=True,drop=True)
    
    return candlesDataBase_BigQuery

#_____

#FUINCIÓN QUE DEVUELVE EL ARREGLO DE LA RESPUESTA DEL MODELO
def variableBuy(candlesDataBase_BigQuery):
    
    global config
    updateConfig("predictionModels.ini")
    periods_ahead = int(get_config("PARAMETERS","PERIODS_AHEAD"))
    
    #_____DETERMINAR SI ESTOY EN BUENA ZONA DE COMPRA
    #for i in range(0,len(candlesDataBase_BigQuery.CLOSE)-int(get_config("PARAMETERS","PERIODS_TO_UTILITY"))):
    #    if (candlesDataBase_BigQuery.CLOSE.values[i]<=np.quantile(list(candlesDataBase_BigQuery.CLOSE.values[i:i+periods_ahead]),0.05)):
    #        contador_utilidad_encontrada=0
    #        for j in range(i+1,i+int(get_config("PARAMETERS","PERIODS_TO_UTILITY"))):
    #            if candlesDataBase_BigQuery.CLOSE[j]/candlesDataBase_BigQuery.CLOSE[i]-1 >= float(get_config("PARAMETERS","MIN_UTILITY")):
    #                contador_utilidad_encontrada=1
    #        if contador_utilidad_encontrada==1:
    #            candlesDataBase_BigQuery.at[i,"BUY"]="SI"
    #        else:
    #            candlesDataBase_BigQuery.at[i,"BUY"]="NO"
    #    else:
    #        candlesDataBase_BigQuery.at[i,"BUY"]="NO"
    
    #_____DETERMINAR SI ESTOY EN BUENA ZONA DE COMPRA
    for i in range(0,len(candlesDataBase_BigQuery.CLOSE)-int(get_config("PARAMETERS","PERIODS_TO_UTILITY"))):
        contador_utilidad_encontrada=0
        for j in range(i+1,i+int(get_config("PARAMETERS","PERIODS_TO_UTILITY"))):
            if candlesDataBase_BigQuery.CLOSE[j]/candlesDataBase_BigQuery.CLOSE[i]-1 >= float(get_config("PARAMETERS","MIN_UTILITY")):
                contador_utilidad_encontrada=1
        if contador_utilidad_encontrada==1:
            candlesDataBase_BigQuery.at[i,"BUY"]="SI"
        else:
            candlesDataBase_BigQuery.at[i,"BUY"]="NO"
            
        #___
        
        if i > 0:
            if candlesDataBase_BigQuery.at[i,"BUY"]=="SI":
                if candlesDataBase_BigQuery.CLOSE[i+1]<candlesDataBase_BigQuery.CLOSE[i]:
                    candlesDataBase_BigQuery.at[i,"BUY"]="NO"
    #___
    
    candlesDataBase_BigQuery.dropna(inplace=True)
    candlesDataBase_BigQuery.reset_index(inplace=True,drop=True)
    candlesDataBase_BigQuery=candlesDataBase_BigQuery[periods_ahead:-int(get_config("PARAMETERS","PERIODS_TO_UTILITY"))]
    candlesDataBase_BigQuery.reset_index(inplace=True,drop=True)
    
    #___
    
    return candlesDataBase_BigQuery

#_____

#FUNCIÓN QUE ESCALA LAS VARIABLES CONTINUAS DE LA BASE DE DATOS
def scaleVariables(candlesDataBase_BigQuery):
    
    global marketDict
    
    #_____ESCALAR VARIABLES
    continuous_variables=list(candlesDataBase_BigQuery.columns)[3:-1]
    marketVariables=list(candlesDataBase_BigQuery.columns)[0:3]
    responseVariable=list(candlesDataBase_BigQuery.columns)[-1:]

    #_____LISTA APPEND DE VARIABLES
    listaAppend=[]

    for i in continuous_variables:

        try:

            scaler=preprocessing.StandardScaler(with_mean=True).fit(np.array(candlesDataBase_BigQuery[[i]]))
            scaled_data=pd.DataFrame(scaler.transform(candlesDataBase_BigQuery[[i]]))
            scaled_data.columns=[i]
            
            marketDict[candlesDataBase_BigQuery.at[0,"MARKET"]][i]={}
            marketDict[candlesDataBase_BigQuery.at[0,"MARKET"]][i]["STD"]=np.std(list(candlesDataBase_BigQuery[[i]].values))
            marketDict[candlesDataBase_BigQuery.at[0,"MARKET"]][i]["MEAN"]=np.mean(list(candlesDataBase_BigQuery[[i]].values))
            
            candlesDataBase_BigQuery[str(i)] = scaled_data[str(i)]
            listaAppend.append(i)

        except:

            pass

    writePickleVariable(marketDict,"marketDict")
    candlesDataBase_BigQuery=candlesDataBase_BigQuery[marketVariables+listaAppend+responseVariable]

    #_____RETURN
    return candlesDataBase_BigQuery

#_____

#FUNCIÓN QUE DEVUELVE LAS COLUMNAS DESPUÉS DE HACER BORUTA, ROLLING, SCALING, BINARY SELECTION, LAGS
def boruta(candlesDataBase_BigQuery):
    
    global CLOSE
    global alpha_boruta
    
    #_____COLUMNAS VARIABLE X
    continuous_variables=list(candlesDataBase_BigQuery.columns)[3:-1]
    
    #_____UPDATE CONFIG PREDICTION
    updateConfig("predictionModels.ini")
    
    if int(get_config("PARAMETERS","BORUTA"))==1:

        #_____VECTOR INPUT
        X=candlesDataBase_BigQuery.copy()
        X=X[continuous_variables]
        my_columns=X.columns
        X=X.values

        #_____VECTOR OUTPUT
        y = candlesDataBase_BigQuery.copy()
        y=y["BUY"].values.ravel()

        #_____RANDOM FOREST BASE PARA LA SELECCIÓN
        rf = RandomForestClassifier(min_samples_split=2,max_features="auto",bootstrap=True,n_jobs=-1,n_estimators=200,max_depth=2,class_weight="balanced")    

        #_____SELECCIONADOR BORUTA
        alpha_boruta=0.05

        boruta_selector = BorutaPy(rf,max_iter=5000,n_estimators=200,alpha=alpha_boruta)
        boruta_selector.fit(X,y)
        features_selected=boruta_selector.n_features_
        selected_columns=list(my_columns[boruta_selector.support_].values)

        if len(selected_columns)==0:
            selected_columns=continuous_variables
            
        writePickleVariable(selected_columns,"variableList")
            
    else:
        
        #selected_columns=continuous_variables
        #selected_columns=ast.literal_eval(get_config("PARAMETERS","BORUTA_COLUMNS"))
        selected_columns=readPickleVariable("variableList")
    
    CLOSE=candlesDataBase_BigQuery.copy()
    CLOSE=CLOSE[["CLOSE"]]
    
    return candlesDataBase_BigQuery[list(candlesDataBase_BigQuery.columns)[0:3]+selected_columns+list(candlesDataBase_BigQuery.columns)[-1:]]

#_____

#FUNCIÓN QUE DEVUELVE EL MODELO DE PREDICCIÓN Y LAS BASES DE DATOS GLOBALES TEST Y TRAIN
def train(candlesDataBase_BigQuery):
    
    global rf_random
    global train_model
    global test_sample
    global train_sample
    
    #_____UPDATE CONFIG PREDICTION
    updateConfig("predictionModels.ini")

    #_____COLUMNAS VARIABLE X
    continuous_variables=list(candlesDataBase_BigQuery.columns)[3:-1]
    
    #_____SEPARAR MUESTRA TRAIN DE TEST, SI SE TRATA DE UNA PRUEBA
    train_sample=candlesDataBase_BigQuery.copy()
    train_sample=train_sample.sample(frac=float(get_config("PARAMETERS","TRAIN_PROPORTION")),random_state=42)
    
    test_sample=candlesDataBase_BigQuery.copy()
    test_sample=test_sample.loc[~test_sample.index.isin(list(train_sample.index.values))]
    
    train_sample.reset_index(inplace=True,drop=True)
    test_sample.reset_index(inplace=True,drop=True)
    
    if int(get_config("PARAMETERS","TRAIN"))==1:

        #_____VECTOR INPUT TRAIN
        X_train=train_sample.copy()
        X_train=X_train[continuous_variables]
        X_train=X_train.values

        #_____VECTOR OUTPUT
        y_train = train_sample.copy()
        y_train=y_train["BUY"].values.ravel()

        #_____RANDOM GRID SEARCH
        # Number of trees in random forest
        #n_estimators = [int(x) for x in np.linspace(start = 100, stop = 2000, num = 3)]
        n_estimators = [2000]
        # Number of features to consider at every split
        #max_features = ['auto', 'sqrt']
        max_features = ['auto']
        # Maximum number of levels in tree
        max_depth = [int(x) for x in np.linspace(2, 10, num = 9)]
        #max_depth.append(None)
        # Minimum number of samples required to split a node
        #min_samples_split = [2, 5, 10]
        min_samples_split = [2]
        # Minimum number of samples required at each leaf node
        #min_samples_leaf = [1, 2, 4]
        min_samples_leaf = [1]
        # Method of selecting samples for training each tree
        #bootstrap = [True, False]
        bootstrap = [True]
        # Create the random grid
        param_grid = {'n_estimators': n_estimators,
                       'max_features': max_features,
                       'max_depth': max_depth,
                       'min_samples_split': min_samples_split,
                       'min_samples_leaf': min_samples_leaf,
                       'class_weight': ["balanced"],
                       'bootstrap': bootstrap}

        #_____RANDOM FOREST DE ENTRENAMIENTO
        rf=RandomForestClassifier(random_state = 42)
        #rf_random = GridSearchCV(estimator = rf, param_grid = param_grid, cv = 10, n_jobs = -1, verbose = 2)
        rf_random = GridSearchCV(estimator = rf, param_grid = param_grid, cv = 10, n_jobs = -1)
        # Fit the random search model
        rf_random.fit(X_train, y_train)

        #_____ENTRENAR MODELOS CON MEJOR PARAMETRIZACIÓN
        train_model = BaggingClassifier(base_estimator=RandomForestClassifier(min_samples_split=rf_random.best_params_["min_samples_split"],
                                                                              max_features=rf_random.best_params_["max_features"],
                                                                              bootstrap=rf_random.best_params_["bootstrap"],
                                                                              n_jobs=-1,
                                                                              n_estimators=rf_random.best_params_["n_estimators"],
                                                                              max_depth=rf_random.best_params_["max_depth"],
                                                                              class_weight=rf_random.best_params_["class_weight"]),random_state=42).fit(X_train, y_train)

        writePickleVariable(train_model,"train_model")
    
#_____

#FUNCIÓN QUE ESCALA LAS VARIABLES DEL TEST CON LOS PARÁMETROS DEL TRAIN (MEDIA + SDT)
def scaleTestVariables(candlesDataBase_BigQuery):
    
    global marketDict
    
    marketDict=readPickleVariable("marketDict")
    
    #_____ESCALAR VARIABLES
    continuous_variables=list(candlesDataBase_BigQuery.columns)[3:-1]
    marketVariables=list(candlesDataBase_BigQuery.columns)[0:3]
    responseVariable=list(candlesDataBase_BigQuery.columns)[-1:]

    #_____LISTA APPEND DE VARIABLES
    listaAppend=[]

    for i in continuous_variables:

        try:

            STD=marketDict[candlesDataBase_BigQuery.at[0,"MARKET"]][i]["STD"]
            MEAN=marketDict[candlesDataBase_BigQuery.at[0,"MARKET"]][i]["MEAN"]
            
            candlesDataBase_BigQuery[str(i)] = candlesDataBase_BigQuery[str(i)]-MEAN
            candlesDataBase_BigQuery[str(i)] = candlesDataBase_BigQuery[str(i)]/STD
            listaAppend.append(i)

        except:

            pass

    candlesDataBase_BigQuery=candlesDataBase_BigQuery[marketVariables+listaAppend+responseVariable]

    #_____RETURN
    return candlesDataBase_BigQuery

#_____

#FUNCIÓN QUE ESCALA LAS VARIABLES DEL TEST CON LOS PARÁMETROS DEL TRAIN (MEDIA + SDT)
def scaleTestVariables_NOBUY(candlesDataBase_BigQuery):
    
    global marketDict
    
    marketDict=readPickleVariable("marketDict")
    
    #_____ESCALAR VARIABLES
    continuous_variables=list(candlesDataBase_BigQuery.columns)[3:-1]
    marketVariables=list(candlesDataBase_BigQuery.columns)[0:3]

    #_____LISTA APPEND DE VARIABLES
    listaAppend=[]

    for i in continuous_variables:

        try:

            STD=marketDict[candlesDataBase_BigQuery.at[0,"MARKET"]][i]["STD"]
            MEAN=marketDict[candlesDataBase_BigQuery.at[0,"MARKET"]][i]["MEAN"]
            
            candlesDataBase_BigQuery[str(i)] = candlesDataBase_BigQuery[str(i)]-MEAN
            candlesDataBase_BigQuery[str(i)] = candlesDataBase_BigQuery[str(i)]/STD
            listaAppend.append(i)

        except:

            pass

    candlesDataBase_BigQuery=candlesDataBase_BigQuery[marketVariables+listaAppend]

    #_____RETURN
    return candlesDataBase_BigQuery
    
#_____

#FUNCIÓN QUE PREDICE SOBRE LOS DATOS TEST
def predictFunction():

    global train_model
    global test_sample
    
    #_____TRAIN_MODEL
    train_model=readPickleVariable("train_model")
    
    #_____COLUMNAS VARIABLE X
    continuous_variables=list(test_sample.columns)[3:-1]
    
    #_____DATOS
    X_test=test_sample.copy()
    X_test=X_test[continuous_variables]
    X_test=X_test.values
    
    #_____PREDICTION
    prediction = train_model.predict(np.array(X_test))
    
    #_____RETURN
    return prediction

#_____

#FUNCIÓN QUE CORRIGE PREDICCIÓN DADO QUE EL PRECIO EN EL QUE ME ENCUENTRO ES MENOR AL PRECIO ANTERIOR
def correctPredict(test_sample,PERIODS_EVALUATION_PAST,MAX_PERIODS_EVALUATION_QUANTILE,PERIODS_LINEAREG_BEFORE):
    
    global myRSI
    
    #_____ACTUALIZAR CONFIG
    updateConfig("/home/ubuntu/SurfNet/predictionModels/PREDICTION/predictionModels.ini")
    
    #_____VARAIABLE QUE PARAMETRIZA PSADO DE EVALUACIÓN
    past_evaluation=PERIODS_EVALUATION_PAST
    
    #_____QUANLIT DE AVIALUACIÓN DE PRECIO MÁXIMO
    max_past_evaluation_quantile=MAX_PERIODS_EVALUATION_QUANTILE
    
    #_____SI RSI ES MAYOR A THRESHOLD, NO COMPRAR
    for i in range(0,len(test_sample)):
        if (test_sample.at[i,"PRED"]=="SI") and (myRSI[i]>float(get_config("PARAMETERS","MY_RSI"))):
            test_sample.at[i,"PRED"]="NO"
    
    #_____RECORRER DATOS DE TEST
    for i in range(past_evaluation,len(test_sample)):
        
        #_____SI LA PREDICCIÓN FUE COMPRAR & SI CLOSE ANTERIOR ES MAYOR
        if (test_sample.at[i,"PRED"]=="SI") and (test_sample.at[i,"CLOSE"]>=np.quantile(test_sample[i-past_evaluation:i].CLOSE.values,max_past_evaluation_quantile)):
            
            #_____CONVERTIR OBSERVACIÓN EN NO COMRPAR
            test_sample.at[i,"PRED"]="NO"
            
    #_____RECORRER DATOS DE TEST
    periods_lineareg=PERIODS_LINEAREG_BEFORE
    for i in range(periods_lineareg,len(test_sample)):
            
        if (test_sample.at[i,"PRED"]=="SI"):

            periods_lineareg=PERIODS_LINEAREG_BEFORE

            X=np.arange(periods_lineareg).reshape(-1, 1)
            Y=test_sample[i-periods_lineareg:i].CLOSE.values.reshape(-1, 1)

            linear_regressor = LinearRegression()
            nuevaVariable=linear_regressor.fit(X, Y)
            Y_pred = linear_regressor.predict(X)

            #_____CORREGIR
            if nuevaVariable.coef_ < 0:

                #_____CONVERTIR OBSERVACIÓN EN NO COMRPAR
                test_sample.at[i,"PRED"]="NO"
                
    #_____REESTRUCTURAR TEST SAMPLE
    test_sample=test_sample[max(past_evaluation,periods_lineareg)+1:]
    test_sample.reset_index(inplace=True,drop=True)
                
    return test_sample

#_____

#FUNCIÓN QUE HACE BACKTESTING PARA DETERMINAR MEJORES PARÁMETROS TOPES DE VENTA
def backtesting(test_sample):
    
    global fractal
    global final_df
    global utility_df
    global train_model
    global tradingBook
    global market_model
    global train_sample
    global backtestingDataFrame
    
    global output_tradingBook
    global output_utility
    
    #_____UPDATE CONFIG PREDICTION
    updateConfig("predictionModels.ini")
    
    #_____INICIAR LISTAS
    TP_LIST=ast.literal_eval(get_config("PARAMETERS","TP_LIST"))
    SL_LIST=ast.literal_eval(get_config("PARAMETERS","SL_LIST"))
    
    #_____INICIAR PARÁMETROS
    PERIODS_EVALUATION_PAST=ast.literal_eval(get_config("PARAMETERS","PERIODS_EVALUATION_PAST"))
    MAX_PERIODS_EVALUATION_QUANTILE=ast.literal_eval(get_config("PARAMETERS","MAX_PERIODS_EVALUATION_QUANTILE"))
    PERIODS_LINEAREG_BEFORE=ast.literal_eval(get_config("PARAMETERS","PERIODS_LINEAREG_BEFORE"))
    
    #_____INICIALIZAR BASE DE DATOS DE SIMULACIÓN DE UTILIDAD
    utility_df=pd.DataFrame()

    #_____LOOP DE SIMULACIÓN
    for past in PERIODS_EVALUATION_PAST:
        for quantile in MAX_PERIODS_EVALUATION_QUANTILE:
            for lineareg in PERIODS_LINEAREG_BEFORE:
                for takeprofit in TP_LIST:
                    for stoploss in SL_LIST:
                        
                        #_____COPIAR BASE DE DATOS
                        test_sample_copy=test_sample.copy()

                        #_____SOBREESCRIBIR VARIABLE CLOSE CON VALORES REALES
                        for back in range(0,len(test_sample_copy)):
                            test_sample_copy.at[back,"CLOSE"]=backtestingDataFrame.loc[backtestingDataFrame.TIME==test_sample_copy.at[back,"TIME"]]["CLOSE"].values[0]
                        
                        #_____CORREGIR PREDICTION
                        test_sample_copy=correctPredict(test_sample_copy,past,quantile,lineareg)
                        predict=test_sample_copy["PRED"]
                        
                        #_____INICIAR PARÁMETROS
                        date=test_sample_copy["TIME"]
                        close_price=test_sample_copy["CLOSE"]
                        
                        #_____CREAR LIBRO DE TRADEO
                        tradingBook=pd.DataFrame(columns=["DATE_BUY","DATE_SELL","NET_BUY_PRICE","NET_SELL_PRICE","STOP_LOSS","NET_UTILITY"])

                        #_____PARÁMETROS DE INICIO
                        j=0
                        ejecuto_compra=0

                        #_____LOOP SEMI-INFINITO
                        while j<=len(close_price)-2 :

                            #_____TRIGGER DE COMPRA
                            if predict[j] == "SI" :
                                buy_authorization = "SI"
                            else:
                                buy_authorization = "NO"

                            #_____SI TRIGGER DE COMPRA SE DISPARA
                            if buy_authorization == "SI":

                                #_____AGREGAR DATOS A LIBRO DE TRADEO + SUMAR UN POCO MÁS DE FEES PARA AJUSTAR A VALORES REALES
                                time_buy=date[j]
                                index_row=len(tradingBook)
                                tradingBook.at[index_row,"DATE_BUY"]=time_buy
                                tradingBook.at[index_row,"DATE_SELL"]=""
                                tradingBook.at[index_row,"NET_BUY_PRICE"]=close_price[j]*(1+0.001)

                                #_____LOOP INFINITO HASTA QUE SE EJECUTE VENTA
                                ejecuto_venta=0 
                                while ejecuto_venta==0 :

                                    #_____SI PRECIO COMIENZA A CAER                
                                    if (ejecuto_venta==0) and j<=(len(close_price)-2):

                                        #_____ACTUALIZAR TIEMPO TRANSCURRIDO
                                        j+=1

                                        #_____ACTUALIZAR MARGEN DE VENTA + SUMAR UN POCO MÁS DE FEES PARA AJUSTAR A VALORES REALES
                                        margen_actual= ((close_price[j]*(1-0.001))/tradingBook.at[index_row,"NET_BUY_PRICE"])-1

                                        #_____VENDER SI ESTOY ARRIBA DE MARGEN DE UTLIDAD
                                        if (ejecuto_venta==0) and (margen_actual >= takeprofit):

                                            #_____EJECUTAR ORDEN DE VENTA
                                            ejecuto_venta=1

                                            #_____ACTUALIZAR BASE DE DATOS TRADINGBOOK_1
                                            tradingBook.at[index_row,"DATE_SELL"]=date[j]
                                            tradingBook.at[index_row,"NET_SELL_PRICE"]=(close_price[j]*(1-0.001))
                                            tradingBook.at[index_row,"STOP_LOSS"]=0
                                            tradingBook.at[index_row,"NET_UTILITY"]=(tradingBook.at[index_row,"NET_SELL_PRICE"]/tradingBook.at[index_row,"NET_BUY_PRICE"])-1

                                        #_____SI PRECIO ESTÁ EN REGIÓN DE STOPLOSS
                                        elif (ejecuto_venta==0) and ((close_price[j] < tradingBook.at[index_row,"NET_BUY_PRICE"]*(1-stoploss+0.001))):

                                            #_____EJECUTAR ORDEN DE VENTA
                                            ejecuto_venta=1

                                            #_____ACTUALIZAR BASE DE DATOS TRADINGBOOK_1
                                            tradingBook.at[index_row,"DATE_SELL"]=date[j]
                                            tradingBook.at[index_row,"NET_SELL_PRICE"]=close_price[j]
                                            tradingBook.at[index_row,"STOP_LOSS"]=1
                                            tradingBook.at[index_row,"NET_UTILITY"]=((close_price[j]*(1-0.001))/tradingBook.at[index_row,"NET_BUY_PRICE"])-1

                                    else:
                                        ejecuto_venta=1

                            else:
                                #_____ACTUALIZAR TIEMPO TRANSCURRIDO
                                j+=1

                        #_____AGREGAR DATOS A BASE DE DATOS DE UTILIDAD
                        index_row=len(utility_df)
                        utility_df.at[index_row,"TP"]=takeprofit
                        utility_df.at[index_row,"SL"]=stoploss
                        utility_df.at[index_row,"PERIODS_EVALUATION_PAST"]=past
                        utility_df.at[index_row,"MAX_PERIODS_EVALUATION_QUANTILE"]=quantile
                        utility_df.at[index_row,"PERIODS_LINEAREG_BEFORE"]=lineareg
                        utility_df.at[index_row,"NET_UTILITY"]=sum(tradingBook.dropna().NET_UTILITY)
                        
    #_____OUTPUT
    output_tradingBook=tradingBook.copy()
    output_utility=utility_df.copy()
    
    #_____FILTRAR BASE DE DATOS POR UTILIDAD MÁS ALTA CONSEGUIDA
    final_df=utility_df.loc[utility_df.NET_UTILITY==max(list(utility_df.NET_UTILITY.values))]
    final_df.reset_index(inplace=True,drop=True)
    final_df=final_df[0:1]
    final_df.at[0,"AVG_NET_UTILITY"]=np.mean(list(utility_df.NET_UTILITY.values))
    final_df.at[0,"MAX_NET_UTILITY_DAY"]=max(list(utility_df.NET_UTILITY.values))/(len(test_sample)*int(fractal)/60/24)
    final_df.at[0,"TRAIN_COLUMNS"]=str(readPickleVariable("variableList"))
    
    #_____TRATAR DE DESCARGAR BASE DE DATOS "EXISTENTE"
    try:
        final_df_PAST=readPickleVariable("/home/ubuntu/SurfNet/predictionModels/MODELS/"+market_model+"_PARAMETERS"+"_"+fractal)
    except:
        
        #_____CREAR BASE DE DATOS INICIAL
        final_df_PAST=pd.DataFrame()
        final_df_PAST.at[0,"TP"]=0.0
        final_df_PAST.at[0,"SL"]=0.0
        final_df_PAST.at[0,"PERIODS_EVALUATION_PAST"]=0.0
        final_df_PAST.at[0,"MAX_PERIODS_EVALUATION_QUANTILE"]=0.0
        final_df_PAST.at[0,"PERIODS_LINEAREG_BEFORE"]=0.0
        final_df_PAST.at[0,"NET_UTILITY"]=0.0
        final_df_PAST.at[0,"AVG_NET_UTILITY"]=0.0
        final_df_PAST.at[0,"MAX_NET_UTILITY_DAY"]=0.0
        final_df_PAST.at[0,"TRAIN_COLUMNS"]=""
        writePickleVariable(final_df_PAST,"/home/ubuntu/SurfNet/predictionModels/MODELS/"+market_model+"_PARAMETERS"+"_"+fractal)

    #_____SI MODELO ACTUAL ES MEJOR QUE MODELO ANTERIOR
    if (final_df.at[0,"NET_UTILITY"]>0) and (final_df.at[0,"AVG_NET_UTILITY"]>0):
        
        if final_df.at[0,"MAX_NET_UTILITY_DAY"]>=float(get_config("PARAMETERS","MINIMUM_UTILITY_DAY")):
        
            #_____ESCRIBIR NUEVOS PARÁMETROS PICKLE
            writePickleVariable(final_df,"/home/ubuntu/SurfNet/predictionModels/MODELS/"+market_model+"_PARAMETERS"+"_"+fractal)
    
#_____
    
#FINCIÓN QUE FILTRA MERCADOS POTENCIALES POR VOLUMEN TRANSADO
def marketList():
    
    #_____CONEXIÓN BINANCE
    binance = ccxt.binance()

    #_____ACTUALIZAR ARCHIVO CONFIG PARA MERCADOS
    updateConfig('/home/ubuntu/SurfNet/candleDataBases/candleDataBases.ini')

    #_____CREAR LISTA DE MERCADOS A EVALUAR
    mercados_Portafolio_Bruto=[]
    mercados=ast.literal_eval(get_config("DATABASES","MARKETS"))

    #_____LISTA VOLUMEN MERCADOS
    marketZeroVolumes_Prop=[]
    marketAverVolumes_List=[]

    #_____LISTA SAFE DE MERCADOS
    safeList=[]
    
    #_____LISTA TOP DE MERCADOS CMC
    cmc_Top_Market_List=topCoinMarketCap()

    #_____RECORRER MERCADOS
    for mercado in mercados:

        #_____DESCARGAR BASE DE DATOS OLCV
        #candlesDataBase_Binance = binance.fetch_ohlcv(mercado+"/USDT","1m",limit=1000)
        #candlesDataBase_Binance = candlesDataBase_Binance[:-1]

        #_____PROPORCIÓN MERCADO MUERTO
        #contador_cero=0
        #contador_aver=0
        #for i in list(range(0,len(candlesDataBase_Binance))):
        #    if candlesDataBase_Binance[i][5] == 0:
        #        contador_cero=contador_cero+1
        #    contador_aver=contador_aver+candlesDataBase_Binance[i][5]

        #marketZeroVolumes_Prop.append(contador_cero/len(candlesDataBase_Binance))
        #marketAverVolumes_List.append(contador_aver/len(candlesDataBase_Binance))

        #_____ACTUALIZAR SAFE LIST
        #if ((contador_cero/len(candlesDataBase_Binance)) == 0):
        #    safeList.append(mercado)
        
        #_____SLEEP
        #time.sleep(2)
        
        #_____AGEREGAR MERCADO SI ESTÁ EN TOP CMC
        if mercado in cmc_Top_Market_List:
            safeList.append(mercado)
        

    #_____NUEVA LISTA DE MERCADOS
    return safeList

#_____

#FUNCIÓN QUE DEVUELVE EL TOG X DE MERCADOS
def topCoinMarketCap():
    
    updateConfig('/home/ubuntu/SurfNet/surfNetRun/semiManual.ini')
    
    API_KEY_CAP="8edb78ea-b23d-4b23-8a6f-5ece6b703720"
    url = 'https://pro-api.coinmarketcap.com/v1/cryptocurrency/listings/latest'
    parameters = {
      'start':'1',
      'limit': get_config("MARKETS","TOP_CMC"),
      'convert':'USD'
    }
    headers = {
      'Accepts': 'application/json',
      'X-CMC_PRO_API_KEY': API_KEY_CAP,
    }

    session = Session()
    session.headers.update(headers)

    try:
      response = session.get(url, params=parameters)
      data = json.loads(response.text)
      #print(data)
    except (ConnectionError, Timeout, TooManyRedirects) as e:
      print(e)
    
    response = pd.DataFrame(data["data"])
    
    return list(response.symbol.values)

#_____

#TWITTER CONNECTION
def OAuth():
    global auth
    API_KEY="gjxBV6ksuiYXxAN8PZ6LmWMZm"
    API_SECRET_KEY="SahRqwCN26NwIF2ZnuWNqVKMHbW8zCBGdQMbfm6qGOu85exbUa"
    ACCESS_TOKEN="1391754949026004993-060MZPsduBwRYKjEeCYCeH5jLg1yZZ"
    ACCESS_TOKEN_SECRET="WZabC59NE88jYVvDxJxNuVp5Q6vu6jRCLn2E2YdrCEGBu"
    #API_KEY="Zb2EpaerwrtVlp7ekZ5QbXD8n"
    #API_SECRET_KEY="3R2jerQ542EAwlo4yjNjvxwGNzXKwUxxTIcvPyPmDH8VUos6ht"
    #ACCESS_TOKEN="1472968680564736010-WEVntWf9cQ8N4ID4MGJUvsRU1A1Pjj"
    #ACCESS_TOKEN_SECRET="sM8tNMLkNIZpuaTNqDRV4VlWsDWwHNpWvYuWAjuZmLpOv"
    
    try:
        auth = tw.OAuthHandler(API_KEY, API_SECRET_KEY)
        auth.set_access_token(ACCESS_TOKEN, ACCESS_TOKEN_SECRET)
        return auth

    except Exception as e:
        return None

# CHECKS

In [8]:
# #_____UPDATE CONFIG PREDICTION
# updateConfig("predictionModels.ini")

# fractal="15"
# market_model="LUNA_USDT_1M"

# #_____CARGAR BASE DE DATOS
# candlesDataBase_BigQuery=downloadDataBaseBigQuery(market_model,time=fractal,rows=int(get_config("PARAMETERS","ROWS_DOWNLOAD")))

# #_____GUARDAR BASE DE DATOS PARA BACKTESTING FUTURO
# backtestingDataFrame=candlesDataBase_BigQuery.copy()

# #_____CREAR VARIABLES INPUT
# candlesDataBase_BigQuery=variableCreation(candlesDataBase_BigQuery)

# #_____CREAR VARIABLE DE RESPUESTA
# candlesDataBase_BigQuery=variableBuy(candlesDataBase_BigQuery)
# candlesDataBase_BigQuery.dropna(inplace=True)
# candlesDataBase_BigQuery.reset_index(inplace=True,drop=True)

# #_____ESCALAR VARIABLES
# # candlesDataBase_BigQuery=scaleVariables(candlesDataBase_BigQuery)
# candlesDataBase_BigQuery=scaleTestVariables(candlesDataBase_BigQuery)

# #_____FEATURE SELECTION
# print(len(list(candlesDataBase_BigQuery.columns)))
# candlesDataBase_BigQuery=boruta(candlesDataBase_BigQuery)
# print(len(list(candlesDataBase_BigQuery.columns)))

# #_____TRAIN MODEL
# train(candlesDataBase_BigQuery)

# #_____PREDECIR VALORES DE TESTEO
# test_sample["PRED"]=predictFunction()

In [9]:
# #_____PARÁMETROS
# candlesDataBase_Graph=test_sample.copy()
# predType="BUY"

# #_____AGREGAR VARIABLE CLOSE
# for i in range(0,len(candlesDataBase_Graph)):
#     candlesDataBase_Graph.at[i,"CLOSE"]=backtestingDataFrame.loc[backtestingDataFrame.TIME==candlesDataBase_Graph.at[i,"TIME"]]["CLOSE"].values[0]

# #_____AGREGAR COLORES
# candlesDataBase_Color=candlesDataBase_Graph.copy()
# candlesDataBase_Color=candlesDataBase_Color[[predType]]
# candlesDataBase_Color=np.where(candlesDataBase_Color[predType] =="SI", "red", "blue")

# #_____DAR TEXTO A EJES
# x_buy = candlesDataBase_Graph[["TIME"]].values
# y_buy = candlesDataBase_Graph[["CLOSE"]].values
# color_buy = candlesDataBase_Color

# #_____RESETEAR
# fig_buy = None

# #_____IMPRIMIR GRÁFICA
# fig_buy = plt.figure(1, figsize=(20,10))
# ax_buy  = fig_buy.add_subplot(111)
# plot_colourline(x_buy,y_buy,color_buy)

In [10]:
# #_____PARÁMETROS
# candlesDataBase_Graph=test_sample.copy()
# predType="PRED"

# #_____AGREGAR VARIABLE CLOSE
# for i in range(0,len(candlesDataBase_Graph)):
#     candlesDataBase_Graph.at[i,"CLOSE"]=backtestingDataFrame.loc[backtestingDataFrame.TIME==candlesDataBase_Graph.at[i,"TIME"]]["CLOSE"].values[0]

# #_____ZOOM
# # candlesDataBase_Graph=candlesDataBase_Graph[len(candlesDataBase_Graph)-400:len(candlesDataBase_Graph)-300]
# # candlesDataBase_Graph=candlesDataBase_Graph[len(candlesDataBase_Graph)-400:]
# # candlesDataBase_Graph.reset_index(inplace=True)

# #_____AGREGAR COLORES
# candlesDataBase_Color=candlesDataBase_Graph.copy()
# candlesDataBase_Color=candlesDataBase_Color[[predType]]
# candlesDataBase_Color=np.where(candlesDataBase_Color[predType] =="SI", "red", "blue")

# #_____DAR TEXTO A EJES
# x_buy = candlesDataBase_Graph[["TIME"]].values
# y_buy = candlesDataBase_Graph[["CLOSE"]].values
# color_buy = candlesDataBase_Color

# #_____RESETEAR
# fig_buy = None

# #_____IMPRIMIR GRÁFICA
# fig_buy = plt.figure(1, figsize=(20,10))
# ax_buy  = fig_buy.add_subplot(111)
# plot_colourline(x_buy,y_buy,color_buy)

# MERGE MARKETS

SELECT MARKETS WITH VOLUME

In [11]:
#_____CCXT
binance = ccxt.binance()

#_____GOOGLE CLOUD CONECTION
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="/home/ubuntu/SurfNet/predictionModels/PREDICTION/bigQueryAccess.json"

# #_____ACTUALIZAR ARCHIVO CONFIG PARA MERCADOS
# updateConfig('/home/ubuntu/SurfNet/candleDataBases/candleDataBases.ini')

# #_____CREAR LISTA DE MERCADOS A EVALUAR
# mercados_Portafolio_Bruto=[]
# mercados=ast.literal_eval(get_config("DATABASES","MARKETS"))

# #_____LISTA VOLUMEN MERCADOS
# marketZeroVolumes_Prop=[]
# marketAverVolumes_List=[]

# #_____LISTA SAFE DE MERCADOS
# safeList=[]

# #_____RECORRER MERCADOS
# for mercado in mercados:

#     #_____DESCARGAR BASE DE DATOS OLCV
#     candlesDataBase_Binance = binance.fetch_ohlcv(mercado+"/USDT","1m",limit=1000)
#     candlesDataBase_Binance = candlesDataBase_Binance[:-1]

#     #_____PROPORCIÓN MERCADO MUERTO
#     contador_cero=0
#     contador_aver=0
#     for i in list(range(0,len(candlesDataBase_Binance))):
#         if candlesDataBase_Binance[i][5] == 0:
#             contador_cero=contador_cero+1
#         contador_aver=contador_aver+candlesDataBase_Binance[i][5]

#     marketZeroVolumes_Prop.append(contador_cero/len(candlesDataBase_Binance))
#     marketAverVolumes_List.append(contador_aver/len(candlesDataBase_Binance))

#     #_____ACTUALIZAR SAFE LIST
#     if ((contador_cero/len(candlesDataBase_Binance)) == 0):
#         safeList.append(mercado)

#     #_____SLEEP
#     time.sleep(1)

# #_____NUEVA LISTA DE MERCADOS
# mercados=safeList

# #_____DICCIONARIO DE MERCADOS
# marketDict={}

MERGE DATAFRAME OF DATA

In [ ]:
# #_____UPDATE CONFIG PREDICTION
# updateConfig("predictionModels.ini")

# #_____TOTAL DATAFRAME
# candlesDataBase_BigQuery_Total=pd.DataFrame()

# for i in mercados:
    
#     fractal="15"
#     market_model=i+"_USDT_1M"

#     #_____CARGAR BASE DE DATOS
#     candlesDataBase_BigQuery=downloadDataBaseBigQuery(market_model,time=fractal,rows=int(get_config("PARAMETERS","ROWS_DOWNLOAD")))

#     #_____FILTRAR POR REGISTROS
#     if len(candlesDataBase_BigQuery)>int(get_config("PARAMETERS","MIN_ROWS")):
#         marketDict[i+"/USDT"]={}
    
#         #_____GUARDAR BASE DE DATOS PARA BACKTESTING FUTURO
#         backtestingDataFrame=candlesDataBase_BigQuery.copy()

#         #_____CREAR VARIABLES INPUT
#         candlesDataBase_BigQuery=variableCreation(candlesDataBase_BigQuery)

#         #_____CREAR VARIABLE DE RESPUESTA
#         candlesDataBase_BigQuery=variableBuy(candlesDataBase_BigQuery)
#         candlesDataBase_BigQuery.dropna(inplace=True)
#         candlesDataBase_BigQuery.reset_index(inplace=True,drop=True)

#         #_____ESCALAR VARIABLES
#         candlesDataBase_BigQuery=scaleVariables(candlesDataBase_BigQuery)

#         #_____APPEND TO BIG DATAFRAME
#         candlesDataBase_BigQuery_Total=candlesDataBase_BigQuery_Total.append(candlesDataBase_BigQuery[:-1000])

#         clear_output()
#         print(len(candlesDataBase_BigQuery_Total))
    
# candlesDataBase_BigQuery_Total_BackUp=candlesDataBase_BigQuery_Total.copy()
# candlesDataBase_BigQuery_Total_BackUp.reset_index(inplace=True,drop=True)

TRAIN

In [ ]:
# #_____BACKUP
# candlesDataBase_BigQuery_Total=candlesDataBase_BigQuery_Total_BackUp.copy()

# #_____FEATURE SELECTION
# print(len(list(candlesDataBase_BigQuery_Total.columns)))
# candlesDataBase_BigQuery_Total=boruta(candlesDataBase_BigQuery_Total)
# print(len(list(candlesDataBase_BigQuery_Total.columns)))

# #_____TRAIN MODEL
# train(candlesDataBase_BigQuery_Total)

PREDICT

In [12]:
marketDict=readPickleVariable("marketDict")
iteration=list(marketDict.keys())

#_____UPDATE CONFIG PREDICTION
updateConfig("predictionModels.ini")

fractal="15"

for itt in iteration:
    print(itt)
    
    #market_model="LUNA/USDT"
    market_model=itt

    #_____CARGAR BASE DE DATOS
    # myData = binance.fetch_ohlcv(market_model,fractal+"m",limit=1000)
    # candlesDataBase_BigQuery =pd.DataFrame(columns=["ID","TIME","MARKET","OPEN","HIGH","LOW","CLOSE","VOLUME"])

    market_model=market_model.replace("/","_")+"_1M"
    candlesDataBase_BigQuery=downloadDataBaseBigQuery(market_model,time=fractal,rows=int(get_config("PARAMETERS","ROWS_DOWNLOAD")))

    # for i in range(0,len(myData)):
    #     candlesDataBase_BigQuery.at[i,"ID"]=myData[i][0]
    #     candlesDataBase_BigQuery.at[i,"TIME"]=datetime.fromtimestamp(myData[i][0]/1000.0)
    #     candlesDataBase_BigQuery.at[i,"MARKET"]=market_model
    #     candlesDataBase_BigQuery.at[i,"OPEN"]=myData[i][1]
    #     candlesDataBase_BigQuery.at[i,"HIGH"]=myData[i][2]
    #     candlesDataBase_BigQuery.at[i,"LOW"]=myData[i][3]
    #     candlesDataBase_BigQuery.at[i,"CLOSE"]=myData[i][4]
    #     candlesDataBase_BigQuery.at[i,"VOLUME"]=myData[i][5]

    #_____GUARDAR BASE DE DATOS PARA BACKTESTING FUTURO
    backtestingDataFrame=candlesDataBase_BigQuery.copy()

    #_____CREAR VARIABLES INPUT
    candlesDataBase_BigQuery=variableCreation(candlesDataBase_BigQuery)

    #_____RSI
    myRSI=list(candlesDataBase_BigQuery.RSI.values)
    myMACD=list(candlesDataBase_BigQuery.MACD.values)
    # myMACD_SHORT=list(candlesDataBase_BigQuery["MACD_"+get_config("PARAMETERS","MACD_SHORT")].values)
    # myMACD_LONG=list(candlesDataBase_BigQuery["MACD_"+get_config("PARAMETERS","MACD_LONG")].values)
    myMACD_TRIGGER=list(candlesDataBase_BigQuery.MACD_TRIGGER.values)

    #_____CREAR VARIABLE DE RESPUESTA
    candlesDataBase_BigQuery=variableBuy(candlesDataBase_BigQuery)
    candlesDataBase_BigQuery.dropna(inplace=True)
    candlesDataBase_BigQuery.reset_index(inplace=True,drop=True)

    #_____ESCALAR VARIABLES
    candlesDataBase_BigQuery=scaleTestVariables(candlesDataBase_BigQuery)

    #_____PREPARACIÓN PARA PREDICCIÓN
    continuous_variables=readPickleVariable("variableList")
    train_model=readPickleVariable("train_model")

    try:

        #_____DATOS
        X_test=candlesDataBase_BigQuery.copy()
        X_test=X_test[continuous_variables]
        X_test=X_test.values

        #_____PREDICTION
        candlesDataBase_BigQuery["PRED"]=train_model.predict(np.array(X_test))

        #_____CORRECT PREDICT
        PERIODS_EVALUATION_PAST=15
        MAX_PERIODS_EVALUATION_QUANTILE=0.95
        PERIODS_LINEAREG_BEFORE=10
        candlesDataBase_BigQuery=correctPredict(candlesDataBase_BigQuery,PERIODS_EVALUATION_PAST,MAX_PERIODS_EVALUATION_QUANTILE,PERIODS_LINEAREG_BEFORE)

        #_____DATE
        past_date = datetime.now()

        #_____BACKTESTING
        fractal="15"
        market_model=market_model.replace("/","_")
        test_sample=candlesDataBase_BigQuery.copy()
        backtesting(test_sample)

        #_____DATE
        future_date = datetime.now()
        difference = (future_date - past_date)
        total_seconds = difference.total_seconds()
        total_seconds

        hola=1

    except:
        hola =0

    if hola==1:

        if len(output_tradingBook)>0:
            for i in range(0,len(output_tradingBook)):
                if i == 0:
                    output_tradingBook.at[i,"MY_UTILITY"]=100*(1+output_tradingBook.at[i,"NET_UTILITY"])
                else:
                    output_tradingBook.at[i,"MY_UTILITY"]=output_tradingBook.at[i-1,"MY_UTILITY"]*(1+output_tradingBook.at[i,"NET_UTILITY"])
            holahola

    clear_output()
    
print(itt,len(output_tradingBook.loc[output_tradingBook.STOP_LOSS==0])/len(output_tradingBook),len(output_tradingBook.loc[output_tradingBook.STOP_LOSS==1])/len(output_tradingBook))
output_tradingBook

ATOM/USDT


Downloading: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 48856/48856 [00:03<00:00, 15443.34rows/s]
/tmp/ipykernel_188142/625403997.py:462: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  candlesDataBase_BigQuery[j+"_1"]=candlesDataBase_BigQuery[j].shift(1)
/tmp/ipykernel_188142/625403997.py:463: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  candlesDataBase_BigQuery[j+"_2"]=candlesDataBase_BigQuery[j].shift(2)
/tmp/ipykernel_188142/625403997.py:464: Perfor

NameError: name 'holahola' is not defined

In [13]:
# print(itt,len(output_tradingBook.loc[output_tradingBook.STOP_LOSS==0])/len(output_tradingBook),len(output_tradingBook.loc[output_tradingBook.STOP_LOSS==1])/len(output_tradingBook))
# output_tradingBook

ATOM/USDT 0.76 0.24


,DATE_BUY,DATE_SELL,NET_BUY_PRICE,NET_SELL_PRICE,STOP_LOSS,NET_UTILITY,MY_UTILITY
0,2022-01-11 04:15:00,2022-01-11 04:30:00,38.25822,38.64132,0,0.010014,101.001353
1,2022-01-11 06:30:00,2022-01-11 10:15:00,38.77874,37.4,1,-0.036518,97.312939
2,2022-01-11 18:15:00,2022-01-11 20:30:00,38.38835,38.81115,0,0.011014,98.384720
3,2022-01-22 11:45:00,2022-01-22 12:15:00,29.70968,30.28968,0,0.019522,100.305412
4,2022-01-22 17:15:00,2022-01-22 18:30:00,29.20918,28.29,1,-0.032437,97.051765
5,2022-01-22 20:30:00,2022-01-22 20:45:00,29.03901,29.53044,0,0.016923,98.694181
6,2022-01-22 22:30:00,2022-01-22 23:15:00,28.88886,29.41056,0,0.018059,100.476486
7,2022-01-23 05:15:00,2022-01-23 05:30:00,30.28025,30.65931,0,0.012518,101.734290
8,2022-01-23 18:15:00,2022-01-23 19:30:00,32.80277,31.54,1,-0.039457,97.720126
9,2022-01-23 21:15:00,2022-01-23 21:30:00,32.29226,32.75721,0,0.014398,99.127118


GRAPH ORIGINAL

In [ ]:
# #_____PARÁMETROS
# candlesDataBase_Graph=candlesDataBase_BigQuery.copy()
# predType="BUY"

# #_____AGREGAR VARIABLE CLOSE
# for i in range(0,len(candlesDataBase_Graph)):
#     candlesDataBase_Graph.at[i,"CLOSE"]=backtestingDataFrame.loc[backtestingDataFrame.TIME==candlesDataBase_Graph.at[i,"TIME"]]["CLOSE"].values[0]

# #_____AGREGAR COLORES
# candlesDataBase_Color=candlesDataBase_Graph.copy()
# candlesDataBase_Color=candlesDataBase_Color[[predType]]
# candlesDataBase_Color=np.where(candlesDataBase_Color[predType] =="SI", "red", "blue")

# #_____DAR TEXTO A EJES
# x_buy = candlesDataBase_Graph[["TIME"]].values
# y_buy = candlesDataBase_Graph[["CLOSE"]].values
# color_buy = candlesDataBase_Color

# #_____RESETEAR
# fig_buy = None

# #_____IMPRIMIR GRÁFICA
# fig_buy = plt.figure(1, figsize=(20,10))
# ax_buy  = fig_buy.add_subplot(111)
# plot_colourline(x_buy,y_buy,color_buy)

GRAPH PREDICT

In [ ]:
# #_____PARÁMETROS
# candlesDataBase_Graph=candlesDataBase_BigQuery.copy()
# predType="PRED"

# #_____AGREGAR VARIABLE CLOSE
# for i in range(0,len(candlesDataBase_Graph)):
#     candlesDataBase_Graph.at[i,"CLOSE"]=backtestingDataFrame.loc[backtestingDataFrame.TIME==candlesDataBase_Graph.at[i,"TIME"]]["CLOSE"].values[0]

# #_____ZOOM
# # candlesDataBase_Graph=candlesDataBase_Graph[len(candlesDataBase_Graph)-400:len(candlesDataBase_Graph)-300]
# # candlesDataBase_Graph=candlesDataBase_Graph[len(candlesDataBase_Graph)-400:]
# # candlesDataBase_Graph.reset_index(inplace=True)

# #_____AGREGAR COLORES
# candlesDataBase_Color=candlesDataBase_Graph.copy()
# candlesDataBase_Color=candlesDataBase_Color[[predType]]
# candlesDataBase_Color=np.where(candlesDataBase_Color[predType] =="SI", "red", "blue")

# #_____DAR TEXTO A EJES
# x_buy = candlesDataBase_Graph[["TIME"]].values
# y_buy = candlesDataBase_Graph[["CLOSE"]].values
# color_buy = candlesDataBase_Color

# #_____RESETEAR
# fig_buy = None

# #_____IMPRIMIR GRÁFICA
# fig_buy = plt.figure(1, figsize=(20,10))
# ax_buy  = fig_buy.add_subplot(111)
# plot_colourline(x_buy,y_buy,color_buy)

# BACKTESTING

In [ ]:
# #_____DATE
# past_date = datetime.now()

# #_____BACKTESTING
# fractal="15"
# market_model=market_model.replace("/","_")
# test_sample=candlesDataBase_BigQuery.copy()
# backtesting(test_sample)

# #_____DATE
# future_date = datetime.now()
# difference = (future_date - past_date)
# total_seconds = difference.total_seconds()
# total_seconds

# RUN - TWITTER

In [ ]:
# #_____LOOP INFINITO
# while True:
    
#     #_____CONEXIÓN BINANCE
#     binance = ccxt.binance()
    
#     #_____MARKET LIST DE MERCADOS POTENCIALES
#     marketsIterationList=marketList()
#     random.shuffle(marketsIterationList)

#     #_____UPDATE CONFIG PREDICTION
#     updateConfig("semiManual.ini")
    
#     #_____MARKETDICT
#     marketDict=readPickleVariable("marketDict")
    
#     #_____FOR LOOP PARA ITERACIÓN DE ALERTA DE MERCADOS
#     for market in marketsIterationList:
        
#         #_____PRINT WITNESS
#         clear_output()
#         print(market+"/USDT",str(datetime.now()-timedelta(hours=5)))

#         #_____PARÁMETROS
#         fractal="15"
#         market_model=market+"/USDT"
        
#         #_____SI MERCADO ESTÁ DENTRO DE MERCADOS EVALUADOS
#         if market_model in marketDict.keys():
        
#             #_____UPDATE CONFIG PREDICTION
#             updateConfig("predictionModels.ini")

#             #_____CARGAR BASE DE DATOS
#             myData = binance.fetch_ohlcv(market_model,fractal+"m",limit=1000)
#             candlesDataBase_BigQuery =pd.DataFrame(columns=["ID","TIME","MARKET","OPEN","HIGH","LOW","CLOSE","VOLUME"])

#             for i in range(0,len(myData)):
#                 candlesDataBase_BigQuery.at[i,"ID"]=myData[i][0]
#                 candlesDataBase_BigQuery.at[i,"TIME"]=datetime.fromtimestamp(myData[i][0]/1000.0)
#                 candlesDataBase_BigQuery.at[i,"MARKET"]=market_model
#                 candlesDataBase_BigQuery.at[i,"OPEN"]=myData[i][1]
#                 candlesDataBase_BigQuery.at[i,"HIGH"]=myData[i][2]
#                 candlesDataBase_BigQuery.at[i,"LOW"]=myData[i][3]
#                 candlesDataBase_BigQuery.at[i,"CLOSE"]=myData[i][4]
#                 candlesDataBase_BigQuery.at[i,"VOLUME"]=myData[i][5]

#             #_____GUARDAR BASE DE DATOS PARA BACKTESTING FUTURO
#             backtestingDataFrame=candlesDataBase_BigQuery.copy()

#             #_____CREAR VARIABLES INPUT
#             candlesDataBase_BigQuery=variableCreation(candlesDataBase_BigQuery)

#             #_____RSI
#             myRSI=list(candlesDataBase_BigQuery.RSI.values)
#             myMACD=list(candlesDataBase_BigQuery.MACD.values)
#             # myMACD_SHORT=list(candlesDataBase_BigQuery["MACD_"+get_config("PARAMETERS","MACD_SHORT")].values)
#             # myMACD_LONG=list(candlesDataBase_BigQuery["MACD_"+get_config("PARAMETERS","MACD_LONG")].values)
#             myMACD_TRIGGER=list(candlesDataBase_BigQuery.MACD_TRIGGER.values)

#             #_____CREAR VARIABLE DE RESPUESTA
#             candlesDataBase_BigQuery=variableBuy(candlesDataBase_BigQuery)
#             candlesDataBase_BigQuery.dropna(inplace=True)
#             candlesDataBase_BigQuery.reset_index(inplace=True,drop=True)

#             #_____ESCALAR VARIABLES
#             candlesDataBase_BigQuery=scaleTestVariables(candlesDataBase_BigQuery)

#             #_____PREPARACIÓN PARA PREDICCIÓN
#             continuous_variables=readPickleVariable("variableList")
#             train_model=readPickleVariable("train_model")

#             #_____DATOS
#             X_test=candlesDataBase_BigQuery.copy()
#             X_test=X_test[continuous_variables]
#             X_test=X_test.values

#             #_____PREDICTION
#             candlesDataBase_BigQuery["PRED"]=train_model.predict(np.array(X_test))

#             #_____CORRECT PREDICT
#             PERIODS_EVALUATION_PAST=15
#             MAX_PERIODS_EVALUATION_QUANTILE=0.95
#             PERIODS_LINEAREG_BEFORE=10
#             candlesDataBase_BigQuery=correctPredict(candlesDataBase_BigQuery,PERIODS_EVALUATION_PAST,MAX_PERIODS_EVALUATION_QUANTILE,PERIODS_LINEAREG_BEFORE)

#             #_____TWITTEAR CUANDO SEA EL CASO
#             if candlesDataBase_BigQuery.at[len(candlesDataBase_BigQuery)-1,"PRED"]=="SI":
#                 while True:
#                     try:
#                         oauth = OAuth()
#                         api = tw.API(oauth)
#                         tweet=api.update_status("¡TRADING ALERT! -> {} -> (SURF)".format(market))
#                         break
#                     except:
#                         time.sleep(10)

#             #_____SLEEP
#             time.sleep(30)
#             clear_output()

# RUN - PARAMETERS

In [ ]:
# #_____CCXT
# binance = ccxt.binance()

# #_____GOOGLE CLOUD CONECTION
# os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="/home/ubuntu/SurfNet/predictionModels/PREDICTION/bigQueryAccess.json"

# while True:

#     #_____NUEVA LISTA DE MERCADOS
#     marketDict=readPickleVariable("marketDict")
#     mercados=list(marketDict.keys())
#     random.shuffle(mercados)
    
#     for market_model in mercados:
        
#         #_____UPDATE CONFIG PREDICTION
#         updateConfig("predictionModels.ini")

#         #_____PARAMETERS
#         fractal="15"
        
#         #_____PRINT
#         clear_output()
#         print(market_model,str(datetime.now()))

#         #_____CARGAR BASE DE DATOS
#         myData = binance.fetch_ohlcv(market_model,fractal+"m",limit=1000)
#         candlesDataBase_BigQuery =pd.DataFrame(columns=["ID","TIME","MARKET","OPEN","HIGH","LOW","CLOSE","VOLUME"])

#         for i in range(0,len(myData)):
#             candlesDataBase_BigQuery.at[i,"ID"]=myData[i][0]
#             candlesDataBase_BigQuery.at[i,"TIME"]=datetime.fromtimestamp(myData[i][0]/1000.0)
#             candlesDataBase_BigQuery.at[i,"MARKET"]=market_model
#             candlesDataBase_BigQuery.at[i,"OPEN"]=myData[i][1]
#             candlesDataBase_BigQuery.at[i,"HIGH"]=myData[i][2]
#             candlesDataBase_BigQuery.at[i,"LOW"]=myData[i][3]
#             candlesDataBase_BigQuery.at[i,"CLOSE"]=myData[i][4]
#             candlesDataBase_BigQuery.at[i,"VOLUME"]=myData[i][5]

#         #_____GUARDAR BASE DE DATOS PARA BACKTESTING FUTURO
#         backtestingDataFrame=candlesDataBase_BigQuery.copy()

#         #_____CREAR VARIABLES INPUT
#         candlesDataBase_BigQuery=variableCreation(candlesDataBase_BigQuery)

#         #_____CREAR VARIABLE DE RESPUESTA
#         candlesDataBase_BigQuery=variableBuy(candlesDataBase_BigQuery)
#         candlesDataBase_BigQuery.dropna(inplace=True)
#         candlesDataBase_BigQuery.reset_index(inplace=True,drop=True)
        
#         #_____RSI
#         myRSI=list(candlesDataBase_BigQuery.RSI.values)

#         #_____ESCALAR VARIABLES
#         candlesDataBase_BigQuery=scaleTestVariables(candlesDataBase_BigQuery)

#         #_____PREPARACIÓN PARA PREDICCIÓN
#         continuous_variables=readPickleVariable("variableList")
#         train_model=readPickleVariable("train_model")

#         #_____DATOS
#         X_test=candlesDataBase_BigQuery.copy()
#         X_test=X_test[continuous_variables]
#         X_test=X_test.values

#         #_____PREDICTION
#         candlesDataBase_BigQuery["PRED"]=train_model.predict(np.array(X_test))

#         #_____BACKTESTING
#         market_model=market_model.replace("/","_")
#         test_sample=candlesDataBase_BigQuery.copy()
#         backtesting(test_sample)
        
#         #_____SLEEP
#         time.sleep(60)

# RUN - PAST

In [ ]:
# #_____CCXT
# binance = ccxt.binance()

# #_____LOOP INFINITO
# while True:
    
#     try:
    
#         #_____GOOGLE CLOUD CONECTION
#         os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="/home/ubuntu/SurfNet/predictionModels/PREDICTION/bigQueryAccess.json"

#         #_____ACTUALIZAR ARCHIVO CONFIG PARA MERCADOS
#         updateConfig('/home/ubuntu/SurfNet/candleDataBases/candleDataBases.ini')

#         #_____CREAR LISTA DE MERCADOS A EVALUAR
#         mercados_Portafolio_Bruto=[]
#         mercados=ast.literal_eval(get_config("DATABASES","MARKETS"))

#         #_____LISTA VOLUMEN MERCADOS
#         marketZeroVolumes_Prop=[]
#         marketAverVolumes_List=[]

#         #_____LISTA SAFE DE MERCADOS
#         safeList=[]

#         #_____RECORRER MERCADOS
#         for mercado in mercados:

#             #_____DESCARGAR BASE DE DATOS OLCV
#             candlesDataBase_Binance = binance.fetch_ohlcv(mercado+"/USDT","1m",limit=1000)
#             candlesDataBase_Binance = candlesDataBase_Binance[:-1]

#             #_____PROPORCIÓN MERCADO MUERTO
#             contador_cero=0
#             contador_aver=0
#             for i in list(range(0,len(candlesDataBase_Binance))):
#                 if candlesDataBase_Binance[i][5] == 0:
#                     contador_cero=contador_cero+1
#                 contador_aver=contador_aver+candlesDataBase_Binance[i][5]

#             marketZeroVolumes_Prop.append(contador_cero/len(candlesDataBase_Binance))
#             marketAverVolumes_List.append(contador_aver/len(candlesDataBase_Binance))

#             #_____ACTUALIZAR SAFE LIST
#             if ((contador_cero/len(candlesDataBase_Binance)) == 0):
#                 safeList.append(mercado)

#             #_____SLEEP
#             time.sleep(2)

#         #_____NUEVA LISTA DE MERCADOS
#         mercados=safeList

#         #_____AGREGAR APELLIDO A MERCADO
#         for i in mercados:
#             mercados_Portafolio_Bruto.append(i+"_USDT_1M")

#         #_____RECORRER LISTA DE MERCADOS SOBRE LOS CUALES REALIZAR MODELOS
#         for market_model in mercados_Portafolio_Bruto:

#             #_____UPDATE CONFIG PREDICTION
#             updateConfig("predictionModels.ini")

#             #_____ITERAR SOBRE FRACTAL
#             for fractal in ast.literal_eval(get_config("PARAMETERS","FRACTAL")):

#                 #_____PRINT
#                 print(market_model,fractal,datetime.now())

#                 #_____CARGAR BASE DE DATOS
#                 candlesDataBase_BigQuery=downloadDataBaseBigQuery(market_model,time=fractal,rows=int(get_config("PARAMETERS","ROWS_DOWNLOAD")))

#                 #_____GUARDAR BASE DE DATOS PARA BACKTESTING FUTURO
#                 backtestingDataFrame=candlesDataBase_BigQuery.copy()

#                 #_____CREAR VARIABLES INPUT
#                 candlesDataBase_BigQuery=variableCreation(candlesDataBase_BigQuery)

#                 #_____CREAR VARIABLE DE RESPUESTA
#                 candlesDataBase_BigQuery=variableBuy(candlesDataBase_BigQuery)

#                 #_____ESCALAR VARIABLES
#                 candlesDataBase_BigQuery=scaleVariables(candlesDataBase_BigQuery)

#                 #_____FILTRAR VARIABLES
#                 candlesDataBase_BigQuery=boruta(candlesDataBase_BigQuery)

#                 #_____ENTRENAR MODELO
#                 train(candlesDataBase_BigQuery,market_model)

#                 #_____PREDECIR VALORES DE TESTEO
#                 test_sample["PRED"]=predictFunction()

#                 #_____BACKTESTING
#                 backtesting(test_sample)

#                 #_____GRAFICAR PREDICCIONES
#                 #graphPredictions_BUY(market_model)

#                 #_____GRAFICAR PREDICCIONES
#                 #graphPredictions_PRED(market_model)

#                 #_____CLEAR OUTPUT
#                 clear_output()

#     except:
#         pass